In [60]:
import pandas as pd
import numpy as np
import os
import deepchem as dc
import pubchempy as pcp
import matplotlib.pyplot as plt
import time
from os.path import dirname, join as pjoin
from deepchem.utils.typing import RDKitMol
from deepchem.feat.base_classes import MolecularFeaturizer
from rdkit import Chem
from rdkit.Chem import AllChem
from numpy.random import random_sample as rs, randint as r
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.model_selection import cross_val_predict as CV
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets, model_selection
from sklearn import svm, model_selection
from sklearn import metrics


In [27]:
def load_init_data(csv_file='./data.csv'):
    loaded_data = pd.read_csv(csv_file)
    n_files = len(loaded_data)
    Ids = loaded_data["mol"]
    labels = loaded_data["pIC50"]
    labels = labels.to_numpy()
    return Ids, labels

In [25]:
def load_round_data(csv_file='./data.csv'):
    loaded_data = pd.read_csv(csv_file)
    n_files = len(loaded_data)
    Ids = loaded_data["Ids"]
    labels = loaded_data["Labels"]
    labels = labels.to_numpy()
    return Ids, labels

In [26]:
def load_featured_data(csv_file):
    loaded_data = pd.read_csv(csv_file)
    samples , feat_count = loaded_data.shape
    label_idx = '%s' %(feat_count - 2)
    features = np.zeros((samples , (feat_count - 2)))
    n_files = len(loaded_data)
    print(loaded_data.shape)
    Ids = loaded_data['0']
    for feat in range(1,(feat_count - 2)):
        features[:, feat] = loaded_data['%s' %feat]
    labels = loaded_data[label_idx]
    labels = labels.to_numpy()
    return Ids, features, labels

In [4]:
def split_data(Ids, Labels):
    X = Ids
    y = Labels
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5)
    train_data = {'Ids':X_train, 'Labels': y_train}
    validation_data = {'Ids':X_val, 'Labels': y_val}
    test_data = {'Ids':X_test, 'Labels': y_test}
    train_data = pd.DataFrame(data=train_data)
    validation_data = pd.DataFrame(data=validation_data)
    test_data = pd.DataFrame(data=test_data)
    train_data.to_csv('BACE_train_dataset.csv')
    validation_data.to_csv('BACE_validation_dataset.csv')
    test_data.to_csv('BACE_test_dataset.csv')

In [5]:
def featurizing_MACKeys(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,169), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    MACKEYS_featurizer = dc.feat.MACCSKeysFingerprint()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:168] = MACKEYS_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('BACE_MACKeys_train_data.csv')

In [49]:
def featurizing_CF(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,2050), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    CF_featurizer = dc.feat.CircularFingerprint()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:2049] = CF_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('BACE_CircularFP_test_data.csv')

In [7]:
def featurizing_PubFP(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,883), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    PubFB_featurizer = dc.feat.PubChemFingerprint()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:882] = PubFB_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('BACE_PubChemFP_train_data.csv')

In [8]:
def featurizing_RDKit(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,202), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    RDKit_featurizer = dc.feat.RDKitDescriptors()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:201] = RDKit_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('BACE_RDKit_train_data.csv')

In [9]:
def KnnLooCV(features, labels, k):
    X = features
    y = labels
    model = KNR(n_neighbors = k)
    pred = CV(model, X, y, cv=len(X))
    r2_score = r2(pred,y)
    print(r2_score)
    return r2_score

In [18]:
def feature_importance(feature_arrays, labels):
    y = labels
    x = feature_arrays
    _, n_features = feature_arrays.shape
    importance_values = np.zeros(n_features)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        x, y, test_size = 0.2, random_state = 1)
    kf = model_selection.KFold(n_splits = 5)
    for train_idx, val_idx in kf.split(x_train):
        feature_imp_model = RandomForestRegressor(
            n_estimators = 1000, criterion="mse",
            min_samples_leaf = 1, max_features = "sqrt")
        feature_imp_model.fit(x_train[train_idx], y_train[train_idx])
        fold_feat_imp = feature_imp_model.feature_importances_
        importance_values += fold_feat_imp
    print(importance_values)
    return importance_values

In [55]:
def feature_rearrage(feature_arrays, importance_values):
    samples, n_features = feature_arrays.shape
    ranked_feature_array = np.zeros((samples, n_features))
    new_order = np.argsort(importance_values)
    count = n_features - 1
    for new_ind in new_order:
        ranked_feature_array[:,count] = feature_arrays[:,new_ind]
        count -= 1
    ranked_features = pd.DataFrame(data=ranked_feature_array)
    ranked_features.to_csv('ranked_featured_CFP_test_data.csv')
    return ranked_feature_array

In [43]:
def SVM_optimization(train_feats, train_labels, val_feats, val_labels):
    train_features = train_feats
    y_train = train_labels
    val_features = val_feats
    y_val = val_labels
    _, total_features = train_features.shape
    kernel = 'rbf'
    lmda = 0.0010
    MSEs = np.zeros(total_features)
    R2s = np.zeros(total_features)
    times = np.zeros(total_features)
    for feats_used in range(1,total_features):
        x_train = train_features[: , 0:feats_used]
        x_val = val_features[: , 0:feats_used]
        prev_time = time.time()
        svc = svm.SVR(kernel=kernel, C=1/lmda)
        svc.fit(x_train, y_train)
        curr_time = time.time()
        y_pred = svc.predict(x_val)
        MSEs[feats_used] = metrics.mean_squared_error(y_val, y_pred)
        R2s[feats_used] = metrics.r2_score(y_val, y_pred)
        times[feats_used] = curr_time - prev_time
    return MSEs, R2s, times

In [58]:
def final_model(train_feats, train_labels, fin_feats, fin_labels):
    train_features = train_feats
    y_train = train_labels
    test_features = fin_feats
    y_test = fin_labels
    _, total_features = train_features.shape
    kernel = 'rbf'
    lmda = 0.0010
    x_train = train_features[: , 0:240]
    x_test = test_features[: , 0:240]
    prev_time = time.time()
    svc = svm.SVR(kernel=kernel, C=1/lmda)
    svc.fit(x_train, y_train)
    curr_time = time.time()
    y_pred = svc.predict(x_test)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    R2 = metrics.r2_score(y_test, y_pred)
    timed = curr_time - prev_time
    return MSE, R2, timed

In [ ]:
Ids_tot, labels_tot = load_init_data('BACE ID and Label.csv')
split_data(Ids_tot, labels_tot)

In [ ]:
Ids_train, labels_train = load_round_data('BACE_train_dataset.csv')
featurizing_MACKeys(Ids_train, labels_train)
featurizing_CF(Ids_train, labels_train)
featurizing_PubFP(Ids_train, labels_train)
featurizing_RDKit(Ids_train, labels_train)

In [16]:
Ids_Mac, features_Mac, labels_Mac = load_featured_data('BACE_MACKeys_train_data.csv')
Ids_CFP, features_CFP, labels_CFP = load_featured_data('BACE_CircularFP_train_data.csv')
Ids_PCFP, features_PCFP, labels_PCFP = load_featured_data('BACE_PubChemFP_train_data.csv')
Ids_RD, features_RD, labels_RD = load_featured_data('BACE_RDKit_train_data.csv')
features_RD_noNaN = np.nan_to_num(features_RD, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 1)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 1)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 1)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 1)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 2)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 2)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 2)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 2)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 3)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 3)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 3)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 3)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 5)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 5)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 5)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 5)

(1059, 170)
(1059, 2051)
(462, 884)
(1059, 203)
0.44359615467138214
0.5201682527989713
0.4313893328124462
0.44699605392342523
0.45992776041542127
0.6101442881752968
0.4108352439574978
0.4341824616046184
0.44267053485826013
0.6061217058321291
0.35850426538659974
0.3944733182333918
0.39838233912613696
0.5740100609463514
0.3082733739291498
0.33698632871423917


In [21]:
importance_values = feature_importance(features_CFP, labels_CFP)

In [42]:
Ids_val, labels_val = load_round_data('BACE_validation_dataset.csv')
featurizing_CF(Ids_val, labels_val)
Ids_CFP_val, features_CFP_val, labels_CFP_val = load_featured_data('BACE_CircularFP_val_data.csv')
ranked_train_feature_array = feature_rearrage(features_CFP, importance_values)

(227, 2051)


In [40]:
ranked_val_feature_array = feature_rearrage(features_CFP_val, importance_values)

In [44]:
MSEs, R2s, times = SVM_optimization(ranked_train_feature_array, labels_CFP, ranked_val_feature_array, labels_CFP_val)

In [56]:
Ids_test, labels_test = load_round_data('BACE_test_dataset.csv')
featurizing_CF(Ids_test, labels_test)
Ids_CFP_test, features_CFP_test, labels_CFP_test = load_featured_data('BACE_CircularFP_test_data.csv')
ranked_test_feature_array = feature_rearrage(features_CFP_test, importance_values)
MSE, R2, time = final_model(ranked_train_feature_array, labels_CFP, ranked_test_feature_array, labels_CFP_test)

(227, 2051)


In [57]:
print(MSE, R2, time)

0.5074651548931994 0.7276166079826318 0.7794139385223389


In [61]:
MSE, R2, time = final_model(ranked_train_feature_array, labels_CFP, ranked_test_feature_array, labels_CFP_test)

In [62]:
print(MSE, R2, time)

0.6683661589297509 0.6412525279352049 0.13863277435302734
